In [1]:
import pandas as pd
import re
from re import match
import requests
from bs4 import BeautifulSoup as soup
import difflib
import math

In [2]:
study_groups = pd.read_csv("dummy_data_sets/study_groups.csv")

In [3]:
study_groups.head()

,Timestamp,UIC Email 📧\nEg. Rchen64@uic.edu,What course do you want to study for?\nEg. CHEM 101\nList additional courses separately using the course options below (not required).,Course 2\n,Course 3\n,Course 4\n,Course 5\n,Please select the residence hall you live in or the commuter option. 🏡,"When studying with other people, I usually act as a(n)________",I’m more of a ______ learner.
0,12/23/2022 23:35:27,rickyyoungtutoring@gmail.com,Chem 444,NaN,NaN,NaN,NaN,Commuter,Teacher,Visual (I understand and remember material mai...
1,1/25/2023 21:47:16,avas@uic.edu,CHEM 314,BIOS 220,NaN,NaN,NaN,James Stukel Towers,Note-taker,"Hands-on (I prefer to touch, move, build, or d..."
2,1/26/2023 13:32:01,carri2@uic.edu,MGMT 340,ENTR 310,BA 220,IDS 355,IDS 200,James Stukel Towers,Listener,"Hands-on (I prefer to touch, move, build, or d..."
3,1/26/2023 13:33:05,Smadan7@uic.edu,Business intelligence and marketing technology,NaN,NaN,NaN,NaN,Thomas Beckham Hall,Facilitator,"Hands-on (I prefer to touch, move, build, or d..."
4,1/26/2023 13:33:55,sahma6@uic.edu,BIOS 352,NaN,NaN,NaN,NaN,Academic and Residential Complex,Listener,Visual (I understand and remember material mai...


In [4]:
study_groups.columns

Index(['Timestamp', 'UIC Email 📧\nEg. Rchen64@uic.edu',
       'What course do you want to study for?\nEg. CHEM 101\nList additional courses separately using the course options below (not required). ',
       'Course 2\n', 'Course 3\n', 'Course 4\n', 'Course 5\n',
       'Please select the residence hall you live in or the commuter option. 🏡',
       'When studying with other people, I usually act as a(n)________',
       'I’m more of a ______ learner. '],
      dtype='object')

In [5]:
study_groups.rename(columns={'UIC Email 📧\nEg. Rchen64@uic.edu':'email', 'What course do you want to study for?\nEg. CHEM 101\nList additional courses separately using the course options below (not required). ':
                            'course_1', 'Course 2\n':'course_2', 'Course 3\n':'course_3', 'Course 4\n':'course_4', 'Course 5\n':'course_5', 'Please select the residence hall you live in or the commuter option. 🏡':'dorm', 
                            'When studying with other people, I usually act as a(n)________':'desired_role', 'I’m more of a ______ learner. ':'desired_learner'}, inplace=True)

In [6]:
study_groups.drop(columns=['Timestamp'], inplace=True)

In [7]:
study_groups['email'] = [row.lower() for row in study_groups['email']]

In [8]:
for col in study_groups[['course_1', 'course_2', 'course_3', 'course_4', 'course_5']]:
    study_groups[str(col)].fillna('NONE', inplace=True)
    study_groups[str(col)] = [row.upper() for row in study_groups[str(col)]]

In [9]:
study_groups = study_groups[study_groups['email'].str.contains('\w*@uic.edu')]

In [10]:
study_groups

,email,course_1,course_2,course_3,course_4,course_5,dorm,desired_role,desired_learner
1,avas@uic.edu,CHEM 314,BIOS 220,NONE,NONE,NONE,James Stukel Towers,Note-taker,"Hands-on (I prefer to touch, move, build, or d..."
2,carri2@uic.edu,MGMT 340,ENTR 310,BA 220,IDS 355,IDS 200,James Stukel Towers,Listener,"Hands-on (I prefer to touch, move, build, or d..."
3,smadan7@uic.edu,BUSINESS INTELLIGENCE AND MARKETING TECHNOLOGY,NONE,NONE,NONE,NONE,Thomas Beckham Hall,Facilitator,"Hands-on (I prefer to touch, move, build, or d..."
4,sahma6@uic.edu,BIOS 352,NONE,NONE,NONE,NONE,Academic and Residential Complex,Listener,Visual (I understand and remember material mai...
5,abang4@uic.edu,ECON 120,MATH 165,HUMAN NUTRITION 196,NONE,NONE,Academic and Residential Complex,Listener,"Reading and Writing (I find writing, reading a..."
...,...,...,...,...,...,...,...,...,...
95,ehamma3@uic.edu,CHEM 124-125,ECON 121,PHYSICS 131,NONE,NONE,James Stukel Towers,Facilitator,Visual (I understand and remember material mai...
96,ichoha4@uic.edu,ACTG 210,IDS 270,IDS 331,FIN 300,MUS 118,Thomas Beckham Hall,Listener,Visual (I understand and remember material mai...
97,cthoma60@uic.edu,BIO 286,NONE,NONE,NONE,NONE,Thomas Beckham Hall,Listener,Visual (I understand and remember material mai...
98,osanu2@uic.edu,CHEM 122,CHEM 123,BIOS 110,ANTHROPOLOGY 100,NONE,Courtyard,Note-taker,Auditory (I learn most effectively by listenin...


In [11]:
html = soup(requests.get(f'https://catalog.uic.edu/ucat/course-descriptions/').content, 'html.parser')

In [12]:
table = html.find('div', attrs = {'class':'sitemap'})

In [13]:
classes = {}
for row in table.find_all('a'):
    classes[row.string.split('(')[0].upper().strip()] = row.string.split('(')[-1].split(')')[0]

In [14]:
classes

{'ACADEMIC SKILLS PROGRAM': 'ASP',
 'ACCOUNTING': 'ACTG',
 'ANATOMY AND CELL BIOLOGY': 'ANAT',
 'ANTHROPOLOGY': 'ANTH',
 'APPLIED HEALTH SCIENCES': 'AHS',
 'ARABIC': 'ARAB',
 'ARCHITECTURE': 'ARCH',
 'ART': 'ART',
 'ART HISTORY': 'AH',
 'BIOCHEMISTRY AND MOLECULAR GENETICS': 'BCMG',
 'BIOLOGICAL SCIENCES': 'BIOS',
 'BIOMEDICAL AND HEALTH INFORMATION SCIENCES': 'BHIS',
 'BIOMEDICAL ENGINEERING': 'BME',
 'BIOPHARMACEUTICAL SCIENCES': 'BPS',
 'BIOSTATISTICS': 'BSTT',
 'BLACK STUDIES': 'BLST',
 'BUSINESS ADMINISTRATION': 'BA',
 'CAMPUS COURSES': 'CC',
 'CATHOLIC STUDIES': 'CST',
 'CENTRAL AND EASTERN EUROPEAN STUDIES': 'CEES',
 'CHEMICAL ENGINEERING': 'CHE',
 'CHEMISTRY': 'CHEM',
 'CHINESE': 'CHIN',
 'CIVIL, MATERIALS, AND ENVIRONMENTAL ENGINEERING': 'CME',
 'CLASSICS': 'CL',
 'COMMUNICATION': 'COMM',
 'COMMUNITY HEALTH SCIENCES': 'CHSC',
 'COMPUTER SCIENCE': 'CS',
 'CRIMINOLOGY, LAW, AND JUSTICE': 'CLJ',
 'CURRICULUM AND INSTRUCTION': 'CI',
 'DESIGN': 'DES',
 'DIALOGUE': 'DLG',
 'DISABILI

In [15]:
study_groups.replace('PSCH', 'PSCH 100', inplace=True)

In [16]:
difflib.get_close_matches('MTH', classes.values())

['MTHT', 'MATH']

In [17]:
def class_list_fix():
    class_list = []
    for i in range(1,6):
        for course in study_groups[f"course_{i}"]:
            class_list.append(course)
    return(class_list)


In [18]:
def patch_class_names(column):
    class_list = class_list_fix()
    for row in study_groups[column]:
        if row == 'NONE':
            pass
        else:
            try:
                cleaned_class = re.search('[A-Z]+', row).group()
                class_num = re.search('\d+', row).group()
                if cleaned_class in classes.values():
                    study_groups.replace(row, classes[cleaned_class] + ' ' + class_num, inplace=True)
                else:
                    pass
                    study_groups.replace(row, str(difflib.get_close_matches(row, class_list)), inplace=True)
            except:
                pass

In [19]:
def clean_class_names(column):
    for row in study_groups[column]:
        if re.match('^[A-Z]* \d{3}$', row):
            pass
        else:
            study_groups.replace(row, 'None', inplace=True)

In [20]:
for i in range(1, 6):
    patch_class_names(f'course_{i}')
    clean_class_names(f'course_{i}')
    study_groups = study_groups[study_groups['course_1'] != 'None'].reset_index(drop = True)

In [21]:
study_groups.replace('CHEM 122123', 'CHEM 123', inplace=True)

In [22]:
study_groups

,email,course_1,course_2,course_3,course_4,course_5,dorm,desired_role,desired_learner
0,avas@uic.edu,CHEM 314,BIOS 220,None,None,None,James Stukel Towers,Note-taker,"Hands-on (I prefer to touch, move, build, or d..."
1,carri2@uic.edu,MGMT 340,ENTR 310,BA 220,IDS 355,IDS 200,James Stukel Towers,Listener,"Hands-on (I prefer to touch, move, build, or d..."
2,sahma6@uic.edu,BIOS 352,None,None,None,None,Academic and Residential Complex,Listener,Visual (I understand and remember material mai...
3,abang4@uic.edu,ECON 120,MATH 165,None,None,None,Academic and Residential Complex,Listener,"Reading and Writing (I find writing, reading a..."
4,mgree32@uic.edu,MATH 125,ACTG 210,None,None,None,James Stukel Towers,Note-taker,"Hands-on (I prefer to touch, move, build, or d..."
...,...,...,...,...,...,...,...,...,...
69,osanu2@uic.edu,CHEM 122,CHEM 123,BIOS 110,None,None,Courtyard,Listener,"Reading and Writing (I find writing, reading a..."
70,ikuku2@uic.edu,CHEM 101,None,None,None,None,Commons West,Facilitator,"Reading and Writing (I find writing, reading a..."
71,mbell37@uic.edu,PHYS 141,CHEM 234,MATH 210,None,None,James Stukel Towers,Listener,"Hands-on (I prefer to touch, move, build, or d..."
72,osanu2@uic.edu,CHEM 122,CHEM 123,BIOS 110,None,None,Courtyard,Note-taker,Auditory (I learn most effectively by listenin...


In [23]:
unique_classes = {}
for i in range(1, 6):
    for row in study_groups[f'course_{i}']:
        if row in unique_classes.keys():
            unique_classes[row] += 1
        else:
            unique_classes[row] = 1

In [24]:
unique_classes

{'CHEM 314': 1,
 'MGMT 340': 3,
 'BIOS 352': 1,
 'ECON 120': 2,
 'MATH 125': 2,
 'CHEM 124': 5,
 'MATH 110': 3,
 'CHEM 234': 3,
 'PSCH 262': 2,
 'CHEM 101': 5,
 'BIOS 110': 8,
 'CHEM 122': 8,
 'CS 141': 6,
 'ACTG 210': 6,
 'MATH 210': 6,
 'MATH 180': 3,
 'CHEM 232': 7,
 'PHYS 141': 5,
 'MATH 181': 4,
 'IDS 410': 1,
 'PPOL 105': 2,
 'MCS 275': 1,
 'PHYS 131': 1,
 'IDS 270': 1,
 'SPAN 102': 1,
 'MGMT 460': 1,
 'MUS 104': 1,
 'MATH 165': 2,
 'DES 209': 1,
 'PSCH 100': 2,
 'PPOL 330': 1,
 'ACTG 355': 1,
 'PHYS 112': 1,
 'CLJ 200': 3,
 'PHYS 441': 1,
 'IDS 371': 1,
 'FR 101': 1,
 'INST 184': 1,
 'MATH 121': 1,
 'ARCH 372': 1,
 'BIOS 350': 2,
 'ACTG 474': 1,
 'CLJ 210': 1,
 'MATH 310': 2,
 'BIOS 220': 3,
 'ENTR 310': 1,
 'None': 175,
 'BIOS 120': 1,
 'EAES 111': 1,
 'ANTH 105': 2,
 'PHYS 132': 3,
 'MATH 220': 1,
 'CS 151': 3,
 'IDS 200': 3,
 'PSCH 242': 2,
 'PHYS 142': 1,
 'US 101': 1,
 'MCS 320': 2,
 'SPAN 103': 2,
 'MGMT 486': 1,
 'ENGL 104': 1,
 'MGMT 453': 2,
 'DES 251': 1,
 'SJ 201': 1,

In [25]:
def generate_groups(class_dict):
    dead_courses = {}
    groups = {}
    for course, count in class_dict.items():
        if course == 'None':
            break
        else:
            list_count = 0
            group_size = 0
            if count % 4 == 3:
                list_count = math.ceil(count / 4)
                group_size = 4
                for i in range(1, int(list_count)):
                    list_name = f'{course}_{i}'
                    globals()[list_name] = []
                    groups[list_name] = int(group_size)
                for i in range(int(list_count), int(list_count) + 1):
                    list_name = f'{course}_{i}'
                    globals()[list_name] = []
                    groups[list_name] = int(group_size) - 1
            else:
                if count % 5 == 0:
                    list_count = math.ceil(count / 5)
                    group_size = 5
                elif count % 4 == 0:
                    list_count = math.ceil(count / 4)
                    group_size = 4
                elif count % 3 == 0:
                    list_count = math.ceil(count / 3)
                    group_size = 3
                else:
                    dead_courses[course] = count

                for i in range(1, int(list_count) + 1):
                    list_name = f'{course}_{i}'
                    globals()[list_name] = []
                    groups[list_name] = int(group_size)
    return groups

In [26]:
groups = generate_groups(unique_classes)
groups

{'MGMT 340_1': 3,
 'CHEM 124_1': 5,
 'MATH 110_1': 3,
 'CHEM 234_1': 3,
 'CHEM 101_1': 5,
 'BIOS 110_1': 4,
 'BIOS 110_2': 4,
 'CHEM 122_1': 4,
 'CHEM 122_2': 4,
 'CS 141_1': 3,
 'CS 141_2': 3,
 'ACTG 210_1': 3,
 'ACTG 210_2': 3,
 'MATH 210_1': 3,
 'MATH 210_2': 3,
 'MATH 180_1': 3,
 'CHEM 232_1': 4,
 'CHEM 232_2': 3,
 'PHYS 141_1': 5,
 'MATH 181_1': 4,
 'CLJ 200_1': 3,
 'BIOS 220_1': 3}

In [35]:
def match_groups(table, groups):
    final_groups = {}
    for course, count in groups.items():
        course = course.split('_')[0]
        temp_group = []
        while count > 0:
            for email in table['email']:
                if (table[table['email'] == email]['course_1'] == course).any():
                    temp_group.append(email)
                    count -= 1
                elif (table[table['email'] == email]['course_2'] == course).any():
                    temp_group.append(email)
                    count -= 1
                elif (table[table['email'] == email]['course_3'] == course).any():
                    temp_group.append(email)
                    count -= 1
                elif (table[table['email'] == email]['course_4'] == course).any():
                    temp_group.append(email)
                    count -= 1
                elif (table[table['email'] == email]['course_5'] == course).any():
                    temp_group.append(email)
                    count -= 1
                else:
                    pass
        final_groups[course] = temp_group
        print(f'{course} is done!')
    return final_groups

In [36]:
final_groups = match_groups(study_groups, groups)

MGMT 340 is done!
CHEM 124 is done!
MATH 110 is done!
CHEM 234 is done!
CHEM 101 is done!
BIOS 110 is done!
BIOS 110 is done!
CHEM 122 is done!
CHEM 122 is done!
CS 141 is done!
CS 141 is done!
ACTG 210 is done!
ACTG 210 is done!
MATH 210 is done!
MATH 210 is done!
MATH 180 is done!
CHEM 232 is done!
CHEM 232 is done!
PHYS 141 is done!
MATH 181 is done!
CLJ 200 is done!
BIOS 220 is done!


In [37]:
with open("study_groups.txt", 'w') as f: 
    for course, emails in final_groups.items(): 
        f.write('%s:%s\n' % (course, emails))